In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)


Quiero identificar que producen los investigadores de una institución. 

Para eso se propone, identificar primero quienes son esos investigadores que declararon como su filiación, la institución que nos interesa en algun de sus publicaciones. 


In [2]:
sat_openalex_work = catalog.load('stg_openalex/sat_openalex_work')
hub_openalex_work = catalog.load('stg_openalex/hub_openalex_work')
hub_openalex_doi = catalog.load('stg_openalex/hub_openalex_doi')

sal_openalex_work = catalog.load('stg_openalex/sal_openalex_work')
hub_openaire_researchproduct = catalog.load('stg_openaire/hub_openaire_researchproduct')
link_openaire_researchproduct_doi = catalog.load('stg_openaire/link_openaire_researchproduct_doi')
sat_openaire_researchproduct = catalog.load('stg_openaire/sat_openaire_researchproduct')

                    INFO     Loading data from stg_openalex/sat_openalex_work                   ]8;id=701853;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=363823;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (SQLTableDataset)...                                                                  

[03/20/25 12:10:50] INFO     Loading data from stg_openalex/hub_openalex_work                   ]8;id=61288;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=353120;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (SQLTableDataset)...                                                                  

                    INFO     Loading data from stg_openalex/hub_openalex_doi                    ]8;id=882857;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=276677;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (SQLTableDataset)...                                                                  

[03/20/25 12:10:51] INFO     Loading data from stg_openalex/sal_openalex_work                   ]8;id=449761;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=410051;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (SQLTableDataset)...                                                                  

[03/20/25 12:10:53] INFO     Loading data from stg_openaire/hub_openaire_researchproduct        ]8;id=592632;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=395756;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (SQLTableDataset)...                                                                  

[03/20/25 12:10:55] INFO     Loading data from stg_openaire/link_openaire_researchproduct_doi   ]8;id=963298;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=709418;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (SQLTableDataset)...                                                                  

[03/20/25 12:10:57] INFO     Loading data from stg_openaire/sat_openaire_researchproduct        ]8;id=455290;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=451806;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (SQLTableDataset)...                                                                  

Merge con OpenAIRE

In [5]:
def get_fact_publication(hub_openalex_work, sat_openalex_work, sal_openalex_work, hub_openalex_doi, link_openaire_researchproduct_doi, sat_openaire_researchproduct):
    fact_publication_openalex = pd.merge(
        hub_openalex_work,
        sat_openalex_work
        ).drop(columns=['load_datetime','source', 'hashdiff'])

    fact_publication = pd.merge(
        fact_publication_openalex[['work_hk','title','type','publication_year','cited_by_count','oa_status']],
        sal_openalex_work[['work_hk','doi_hk','mag_hk']]
    )

    fact_publication = pd.merge(
        fact_publication,
        hub_openalex_doi[['doi_hk','doi']],
        how='left'
    )

    fact_publication = pd.merge(
        fact_publication,
        link_openaire_researchproduct_doi,
        on="doi_hk",
        how='left'
    )

    fact_publication = pd.merge(
        fact_publication,
        sat_openaire_researchproduct[['researchproduct_hk','citation_count','citation_class','impulse','impulse_class','influence','influence_class','popularity','popularity_class','downloads','views']],
        on="researchproduct_hk",
        how='left'
    )

    fact_publication.rename(columns={'citation_count': 'citation_count_openaire'}, inplace=True)
    fact_columns = [
        'title',
        'doi',
        'publication_year',
        'type',
        'cited_by_count',
        'oa_status',
        'citation_count_openaire',
        'downloads',
        'views'
    ]

    fact_publication = fact_publication[fact_columns].sort_values('cited_by_count', ascending=False)
    return fact_publication


In [6]:
fact_publication = get_fact_publication(hub_openalex_work, sat_openalex_work, sal_openalex_work, hub_openalex_doi, link_openaire_researchproduct_doi, sat_openaire_researchproduct)
fact_publication

,title,doi,publication_year,type,cited_by_count,oa_status,citation_count_openaire,downloads,views
23352,The ATLAS Experiment at the CERN Large Hadron ...,10.1088/1748-0221/3/08/s08003,2008,article,3806,green,NaN,NaN,NaN
36220,The Leiden/Argentine/Bonn (LAB) Survey of Gala...,10.1051/0004-6361:20041864,2005,article,3199,bronze,3183.0,87.0,15.0
32916,The clustering of galaxies in the completed SD...,10.1093/mnras/stx721,2017,article,2582,bronze,2255.0,47.0,12.0
11634,Genome sequence and analysis of the tuber crop...,10.1038/nature10158,2011,article,1979,hybrid,1847.0,736.0,628.0
15729,The Simons Genome Diversity Project: 300 genom...,10.1038/nature18964,2016,article,1474,green,1177.0,45.0,18.0
...,...,...,...,...,...,...,...,...,...
17430,"Violencia, memoria y género. Gabriela Cabezón ...",10.24215/18533701e068,2020,article,0,gold,0.0,69.0,59.0
17432,Análisis de la descarga submarina en dos secto...,10.30550/j.agl/2023.34.2/1838,2023,article,0,hybrid,NaN,NaN,NaN
17433,57Fe-Mössbauer spectroscopic study of some dou...,10.1016/j.matlet.2024.136151,2024,article,0,closed,NaN,NaN,NaN
39556,Influencia del ambiente cultural en la generac...,10.5281/zenodo.5894916,2022,article,0,green,0.0,0.0,0.0
